In [1]:
# ---------------------------------------------------------
# Setup & Imports
# ---------------------------------------------------------
from ultralytics import YOLO
from pathlib import Path
from torchinfo import summary

In [2]:
# ---------------------------------------------------------
#  Define project root (main folder)
# ---------------------------------------------------------
project_root = Path("..").resolve()
print("Project Root:", project_root)

Project Root: D:\tracking\YOLO-TWEAKS


In [3]:
# ---------------------------------------------------------
# Load a YAML YOLO model 
# ---------------------------------------------------------
model1 = YOLO("../tweaks/v8v10.yaml") 
print("Model Loaded Successfully ✅")
# ---------------------------------------------------------

WARNING no model scale passed. Assuming scale='n'.
Model Loaded Successfully ✅


In [10]:
model1_summary = summary(model1.model, input_size=(1, 3, 320, 320))
print(model1_summary)

Layer (type:depth-idx)                                       Output Shape              Param #
DetectionModel                                               [1, 300, 6]               --
├─Sequential: 1-1                                            --                        --
│    └─Conv: 2-1                                             [1, 16, 160, 160]         --
│    │    └─Conv2d: 3-1                                      [1, 16, 160, 160]         432
│    │    └─BatchNorm2d: 3-2                                 [1, 16, 160, 160]         32
│    └─v10Detect: 2-100                                      --                        (recursive)
│    │    └─ModuleList: 3-128                                --                        (recursive)
│    └─Conv: 2-3                                             [1, 32, 80, 80]           --
│    │    └─Conv2d: 3-4                                      [1, 32, 80, 80]           4,608
│    │    └─BatchNorm2d: 3-5                                 [1, 32, 80, 

In [ ]:
# Train the model on the dataset
model1.train(
    data=str(project_root / "AgriPest" / "data.yaml"),  # Dataset YAML
    imgsz=320,                                          # Input image size
    epochs=50,                                          # Total training epochs
    batch=24,                                           # Training batch size
    project=str(project_root),                          # Saves outputs at root
    name="./experiments/v8v10/train",                       # Folder for experiment

    pretrained=True,                                    # Uses pretrained weights
    device="cpu",                                       # Use CPU training

    # Data augmentation (built-in YOLO flags)
    augment=True,                                       # Enables default augmentations

    # Training controls
    patience=5,                                        # Early stopping patience
    warmup_epochs=5,                                    # Warmup period before full training
)

# 735m 45.5s

New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.220  Python-3.10.19 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-12500)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\tracking\YOLO-TWEAKS\AgriPest\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../tweaks/v8v10.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000212D049BF40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.04504

In [15]:
# ---------------------------------------------------------
# Validate the trained model
# ---------------------------------------------------------
print("\nStarting Model Validation...")

# ---------------------------------------------------------
# Load the best trained model for validation
# ---------------------------------------------------------
best_model_path1 = project_root / "experiments" / "v8v10" / "train" / "weights" / "best.pt"
best_model_val1 = YOLO(str(best_model_path1))
print("Best Model for Validation Loaded Successfully")


Starting Model Validation...
Best Model for Validation Loaded Successfully


In [16]:
# ---------------------------------------------------------
# Validate the model on the 'val' split
# ---------------------------------------------------------
metrics1 = best_model_val1.val(
    data=str(project_root / "AgriPest" / "data.yaml"),  # Dataset YAML
    imgsz=320,
    batch=24,
    split="val",  # Use the validation set
    project=str(project_root),
    # Save validation results to the requested folder
    name="./experiments/v8v10/valid",
    device="cpu",                                       # Use CPU for validation     
    
    # Validation settings (usually default is fine)
    iou=0.6,                                          # IoU threshold for NMS
    conf=0.25,                                        # Confidence threshold
)
print("Validation Metrics:", metrics1)
# ---------------------------------------------------------

Ultralytics 8.3.220  Python-3.10.19 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-12500)
v8v10 summary (fused): 85 layers, 2,472,820 parameters, 0 gradients, 6.7 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 34.511.4 MB/s, size: 38.6 KB)
val: Scanning D:\tracking\YOLO-TWEAKS\AgriPest\valid\labels.cache... 1095 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1095/1095 1.3Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 46/46 2.1it/s 22.1s0.5ss
                   all       1095       1341      0.656      0.513      0.608       0.39
                  Ants         96        178      0.587      0.416      0.497      0.226
                  Bees         99        110      0.756      0.591      0.699      0.347
               Beetles         89        100      0.571       0.28      0.431      0.254
          Caterpillars         77        139      0.618      0.151      0.382       0.22
            Earthworms     

In [17]:
# ---------------------------------------------------------
# Load the best trained model for inference (Prediction)
# ---------------------------------------------------------
# Re-use best_model_val or load again if in a separate script
best_model_pred1 = YOLO(str(best_model_path1))
print("Best Model for Inference Loaded Successfully ✅")

# ---------------------------------------------------------
# Run inference on the 'test' dataset
# ---------------------------------------------------------
# *Note: The 'split' argument is for .val() and is not used in .predict().
# We use 'source' to point to the directory containing test images.*

results1 = best_model_pred1.predict(
    # Assuming the 'test' images folder is here based on the data.yaml structure
    source=str(project_root / "AgriPest"  / "test" / "images"),
    imgsz=320,
     batch=24,
    save=True,  
    project=str(project_root),
    name="experiments/v8v10/predict",
    device="cpu"
)
# ---------------------------------------------------------

Best Model for Inference Loaded Successfully ✅

image 1/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-100-_jpg.rf.830b8949c165c9eb35fcd0e2c6e9f9c5.jpg: 320x320 1 Caterpillars, 12.3ms
image 2/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-105-_jpg.rf.20edce3cd29328ca9b85276c74a19544.jpg: 320x320 1 Weevils, 12.3ms
image 3/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-112-_jpg.rf.c80d31c33349fc0c6ab7902c051e9e2e.jpg: 320x320 1 Weevils, 12.3ms
image 4/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-118-_jpg.rf.7aa454ca9653a7435849b6a6b4bd468c.jpg: 320x320 1 Weevils, 12.3ms
image 5/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-13-_jpg.rf.6a0193d603a8a580f6513bc99ae0f456.jpg: 320x320 1 Weevils, 12.3ms
image 6/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-138-_jpg.rf.c20651f33431db68042f6bc4dc0d7df3.jpg: 320x320 2 Weevilss, 12.3ms
image 7/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-155-_jpg.rf.4242ca36e3444f70f2ab7acfbc3f2d5

In [5]:
# ---------------------------------------------------------
# Load a YAML YOLO model 
# ---------------------------------------------------------
model2 = YOLO("../tweaks/v10v11.yaml") 
print("Model Loaded Successfully ✅")
# ---------------------------------------------------------

WARNING no model scale passed. Assuming scale='n'.
Model Loaded Successfully ✅


In [9]:
model2_summary = summary(model2.model, input_size=(1, 3, 320, 320))
print(model2_summary)

Layer (type:depth-idx)                                       Output Shape              Param #
DetectionModel                                               [1, 84, 2100]             --
├─Sequential: 1-1                                            --                        --
│    └─Conv: 2-1                                             [1, 16, 160, 160]         --
│    │    └─Conv2d: 3-1                                      [1, 16, 160, 160]         432
│    │    └─BatchNorm2d: 3-2                                 [1, 16, 160, 160]         32
│    └─Detect: 2-118                                         --                        (recursive)
│    │    └─ModuleList: 3-140                                --                        (recursive)
│    └─Conv: 2-3                                             [1, 32, 80, 80]           --
│    │    └─Conv2d: 3-4                                      [1, 32, 80, 80]           4,608
│    │    └─BatchNorm2d: 3-5                                 [1, 32, 80, 

In [ ]:
# Train the model on the dataset
model2.train(
    data=str(project_root / "AgriPest" / "data.yaml"),  # Dataset YAML
    imgsz=320,                                          # Input image size
    epochs=50,                                          # Total training epochs
    batch=24,                                           # Training batch size
    project=str(project_root),                          # Saves outputs at root
    name="./experiments/v10v11/train",                       # Folder for experiment

    pretrained=True,                                    # Uses pretrained weights
    device="cpu",                                       # Use CPU training

    # Data augmentation (built-in YOLO flags)
    augment=True,                                       # Enables default augmentations

    # Training controls
    patience=5,                                        # Early stopping patience
    warmup_epochs=5,                                    # Warmup period before full training
)

# 962m 50.5s

New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.220  Python-3.10.19 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-12500)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\tracking\YOLO-TWEAKS\AgriPest\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../tweaks/v10v11.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000212D9FA9EA0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.04504

In [19]:
# ---------------------------------------------------------
# Validate the trained model
# ---------------------------------------------------------
print("\nStarting Model Validation...")

# ---------------------------------------------------------
# Load the best trained model for validation
# ---------------------------------------------------------
best_model_path2 = project_root / "experiments" / "v10v11" / "train" / "weights" / "best.pt"
best_model_val2 = YOLO(str(best_model_path2))
print("Best Model for Validation Loaded Successfully")


Starting Model Validation...
Best Model for Validation Loaded Successfully


In [ ]:
# ---------------------------------------------------------
# Validate the model on the 'val' split
# ---------------------------------------------------------
metrics2 = best_model_val2.val(
    data=str(project_root / "AgriPest" / "data.yaml"),  # Dataset YAML
    imgsz=320,
    batch=24,
    split="val",  # Use the validation set
    project=str(project_root),
    # Save validation results to the requested folder
    name="./experiments/v10v11/valid",
    device="cpu",                                       # Use CPU for validation     
    
    # Validation settings (usually default is fine)
    iou=0.6,                                          # IoU threshold for NMS
    conf=0.25,                                        # Confidence threshold
)
print("Validation Metrics:", metrics2)
# ---------------------------------------------------------
# 38.3s

Ultralytics 8.3.220  Python-3.10.19 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-12500)
v10v11 summary (fused): 102 layers, 3,746,788 parameters, 0 gradients, 13.7 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 61.040.4 MB/s, size: 39.9 KB)
val: Scanning D:\tracking\YOLO-TWEAKS\AgriPest\valid\labels.cache... 1095 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1095/1095 546.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 46/46 1.2it/s 36.9s0.8ss
                   all       1095       1341      0.671      0.591      0.655      0.409
                  Ants         96        178      0.649      0.499      0.577      0.245
                  Bees         99        110      0.732       0.72      0.762      0.376
               Beetles         89        100      0.556       0.41      0.465      0.249
          Caterpillars         77        139      0.395      0.183       0.29      0.162
            Earthworms

In [ ]:
# ---------------------------------------------------------
# Load the best trained model for inference (Prediction)
# ---------------------------------------------------------
# Re-use best_model_val or load again if in a separate script
best_model_pred2 = YOLO(str(best_model_path2))
print("Best Model for Inference Loaded Successfully ✅")

# ---------------------------------------------------------
# Run inference on the 'test' dataset
# ---------------------------------------------------------
# *Note: The 'split' argument is for .val() and is not used in .predict().
# We use 'source' to point to the directory containing test images.*

results2 = best_model_pred2.predict(
    # Assuming the 'test' images folder is here based on the data.yaml structure
    source=str(project_root / "AgriPest"  / "test" / "images"),
    imgsz=320,
     batch=24,
    save=True,  
    project=str(project_root),
    name="experiments/v10v11/predict",
    device="cpu"
)
# ---------------------------------------------------------
# 19.7s

Best Model for Inference Loaded Successfully ✅

image 1/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-100-_jpg.rf.830b8949c165c9eb35fcd0e2c6e9f9c5.jpg: 320x320 1 Weevils, 24.4ms
image 2/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-105-_jpg.rf.20edce3cd29328ca9b85276c74a19544.jpg: 320x320 1 Weevils, 24.4ms
image 3/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-112-_jpg.rf.c80d31c33349fc0c6ab7902c051e9e2e.jpg: 320x320 1 Weevils, 24.4ms
image 4/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-118-_jpg.rf.7aa454ca9653a7435849b6a6b4bd468c.jpg: 320x320 1 Weevils, 24.4ms
image 5/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-13-_jpg.rf.6a0193d603a8a580f6513bc99ae0f456.jpg: 320x320 1 Weevils, 24.4ms
image 6/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-138-_jpg.rf.c20651f33431db68042f6bc4dc0d7df3.jpg: 320x320 1 Weevils, 24.4ms
image 7/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-155-_jpg.rf.4242ca36e3444f70f2ab7acfbc3f2d5e.jpg:

In [22]:
# ---------------------------------------------------------
# Load a YAML YOLO model 
# ---------------------------------------------------------
model3 = YOLO("../tweaks/v11v8.yaml") 
print("Model Loaded Successfully ✅")
# ---------------------------------------------------------

WARNING no model scale passed. Assuming scale='n'.
Model Loaded Successfully ✅


In [23]:
model3_summary = summary(model3.model, input_size=(1, 3, 320, 320))
print(model3_summary)

Layer (type:depth-idx)                                       Output Shape              Param #
DetectionModel                                               [1, 84, 2100]             --
├─Sequential: 1-1                                            --                        --
│    └─Conv: 2-1                                             [1, 16, 160, 160]         --
│    │    └─Conv2d: 3-1                                      [1, 16, 160, 160]         432
│    │    └─BatchNorm2d: 3-2                                 [1, 16, 160, 160]         32
│    └─Detect: 2-114                                         --                        (recursive)
│    │    └─ModuleList: 3-137                                --                        (recursive)
│    └─Conv: 2-3                                             [1, 32, 80, 80]           --
│    │    └─Conv2d: 3-4                                      [1, 32, 80, 80]           4,608
│    │    └─BatchNorm2d: 3-5                                 [1, 32, 80, 

In [ ]:
# Train the model on the dataset
model3.train(
    data=str(project_root / "AgriPest" / "data.yaml"),  # Dataset YAML
    imgsz=320,                                          # Input image size
    epochs=50,                                          # Total training epochs
    batch=24,                                           # Training batch size
    project=str(project_root),                          # Saves outputs at root
    name="./experiments/v11v8/train",                       # Folder for experiment

    pretrained=True,                                    # Uses pretrained weights
    device="cpu",                                       # Use CPU training

    # Data augmentation (built-in YOLO flags)
    augment=True,                                       # Enables default augmentations

    # Training controls
    patience=5,                                        # Early stopping patience
    warmup_epochs=5,                                    # Warmup period before full training
)

# 843m 42.0s
# ---------------------------------------------------------

New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.220  Python-3.10.19 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-12500)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\tracking\YOLO-TWEAKS\AgriPest\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../tweaks/v11v8.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000212904B09A0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.04504

In [25]:
# ---------------------------------------------------------
# Validate the trained model
# ---------------------------------------------------------
print("\nStarting Model Validation...")

# ---------------------------------------------------------
# Load the best trained model for validation
# ---------------------------------------------------------
best_model_path3 = project_root / "experiments" / "v11v8" / "train" / "weights" / "best.pt"
best_model_val3 = YOLO(str(best_model_path3))
print("Best Model for Validation Loaded Successfully")


Starting Model Validation...
Best Model for Validation Loaded Successfully


In [ ]:
# ---------------------------------------------------------
# Validate the model on the 'val' split
# ---------------------------------------------------------
metrics3 = best_model_val3.val(
    data=str(project_root / "AgriPest" / "data.yaml"),  # Dataset YAML
    imgsz=320,
    batch=24,
    split="val",  # Use the validation set
    project=str(project_root),
    # Save validation results to the requested folder
    name="./experiments/v11v8/valid",
    device="cpu",                                       # Use CPU for validation     
    
    # Validation settings (usually default is fine)
    iou=0.6,                                          # IoU threshold for NMS
    conf=0.25,                                        # Confidence threshold
)
print("Validation Metrics:", metrics3)
# ---------------------------------------------------------
# 32.8s

Ultralytics 8.3.220  Python-3.10.19 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-12500)
v11v8 summary (fused): 95 layers, 4,209,736 parameters, 0 gradients, 13.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 72.043.3 MB/s, size: 39.9 KB)
val: Scanning D:\tracking\YOLO-TWEAKS\AgriPest\valid\labels.cache... 1095 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1095/1095  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 46/46 1.5it/s 31.4s0.7ss
                   all       1095       1341      0.642      0.552      0.631      0.384
                  Ants         96        178      0.598      0.455      0.534       0.25
                  Bees         99        110      0.717        0.7      0.778      0.388
               Beetles         89        100      0.573       0.36      0.478      0.229
          Caterpillars         77        139      0.468      0.151      0.322      0.174
            Earthworms         53 

In [ ]:
# ---------------------------------------------------------
# Load the best trained model for inference (Prediction)
# ---------------------------------------------------------
# Re-use best_model_val or load again if in a separate script
best_model_pred3 = YOLO(str(best_model_path3))
print("Best Model for Inference Loaded Successfully ✅")

# ---------------------------------------------------------
# Run inference on the 'test' dataset
# ---------------------------------------------------------
# *Note: The 'split' argument is for .val() and is not used in .predict().
# We use 'source' to point to the directory containing test images.*

results3 = best_model_pred3.predict(
    # Assuming the 'test' images folder is here based on the data.yaml structure
    source=str(project_root / "AgriPest"  / "test" / "images"),
    imgsz=320,
     batch=24,
    save=True,  
    project=str(project_root),
    name="experiments/v11v8/predict",
    device="cpu"
)
# ---------------------------------------------------------
# 16.9s

Best Model for Inference Loaded Successfully ✅

image 1/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-100-_jpg.rf.830b8949c165c9eb35fcd0e2c6e9f9c5.jpg: 320x320 1 Weevils, 20.1ms
image 2/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-105-_jpg.rf.20edce3cd29328ca9b85276c74a19544.jpg: 320x320 1 Weevils, 20.1ms
image 3/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-112-_jpg.rf.c80d31c33349fc0c6ab7902c051e9e2e.jpg: 320x320 1 Weevils, 20.1ms
image 4/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-118-_jpg.rf.7aa454ca9653a7435849b6a6b4bd468c.jpg: 320x320 1 Weevils, 20.1ms
image 5/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-13-_jpg.rf.6a0193d603a8a580f6513bc99ae0f456.jpg: 320x320 1 Weevils, 20.1ms
image 6/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-138-_jpg.rf.c20651f33431db68042f6bc4dc0d7df3.jpg: 320x320 2 Weevilss, 20.1ms
image 7/546 D:\tracking\YOLO-TWEAKS\AgriPest\test\images\Weevil-155-_jpg.rf.4242ca36e3444f70f2ab7acfbc3f2d5e.jpg